In [ ]:
#!pip install skll

In [2]:
import numpy as np
import pandas as pd
import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

np.random.seed(1337)

import tensorflow as tf

# config = tf.ConfigProto(device_count={"CPU": 8,"GPU":0})
# keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8410855416115645961
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1605077269828070216
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16507576182715105756
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7906669364
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17337649548598762880
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
def split_data(X, y, test_data_size):
    """
    Split data into test and training datasets.

    INPUT
        X: NumPy array of arrays
        y: Pandas series, which are the labels for input array X
        test_data_size: size of test/train split. Value from 0 to 1

    OUPUT
        Four arrays: X_train, X_test, y_train, and y_test
    """
    return train_test_split(X, y, test_size=test_data_size, random_state=42)


def reshape_data(arr, img_rows, img_cols, channels):
    """
    Reshapes the data into format for CNN.

    INPUT
        arr: Array of NumPy arrays.
        img_rows: Image height
        img_cols: Image width
        channels: Specify if the image is grayscale (1) or RGB (3)

    OUTPUT
        Reshaped array of NumPy arrays.
    """
    return arr.reshape(arr.shape[0], img_rows, img_cols, channels)


def cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus):
    """
    Define and run the Convolutional Neural Network

    INPUT
        X_train: Array of NumPy arrays
        X_test: Array of NumPy arrays
        y_train: Array of labels
        y_test: Array of labels
        kernel_size: Initial size of kernel
        nb_filters: Initial number of filters
        channels: Specify if the image is grayscale (1) or RGB (3)
        nb_epoch: Number of epochs
        batch_size: Batch size for the model
        nb_classes: Number of classes for classification

    OUTPUT
        Fitted CNN model
    """

    model = Sequential()

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     strides=1,
                     input_shape=(img_rows, img_cols, channels), activation="relu"))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), activation="relu"))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), activation="relu"))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    print("Model flattened out to: ", model.output_shape)

    model.add(Dense(128))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.25))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    #model = multi_gpu_model(model, gpus=nb_gpus)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    stop = EarlyStopping(monitor='val_acc',
                         min_delta=0.001,
                         patience=2,
                         verbose=0,
                         mode='auto')

    tensor_board = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

    model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
              verbose=1,
              validation_split=0.2,
              class_weight='auto',
              callbacks=[stop, tensor_board])

    return model


def save_model(model, score, model_name):
    """
    Saves Keras model to an h5 file, based on precision_score

    INPUT
        model: Keras model object to be saved
        score: Score to determine if model should be saved.
        model_name: name of model to be saved
    """

    if score >= 0.75:
        print("Saving Model")
        model.save("models/" + model_name + "_recall_" + str(round(score, 4)) + ".h5")
    else:
        print("Model Not Saved.  Score: ", score)

In [15]:
# Specify parameters before model is run.
batch_size = 64

nb_classes = 2
nb_epoch = 30

img_rows, img_cols = 256, 256
channels = 3
nb_filters = 32
kernel_size = (8, 8)

In [6]:
# Import data
labels = pd.read_csv("0X_trainLabels_master_256_v2.csv")
X = np.load("0X_train.npy")
y = np.array([1 if l >= 1 else 0 for l in labels['level']])
# y = np.array(labels['level'])

In [7]:
print("Splitting data into test/ train datasets")
X_train, X_test, y_train, y_test = split_data(X, y, 0.2)

Splitting data into test/ train datasets


In [8]:
del X
del y

In [9]:
print("Reshaping Data")
X_train = reshape_data(X_train, img_rows, img_cols, channels)
X_test = reshape_data(X_test, img_rows, img_cols, channels)

print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)

Reshaping Data
X_train Shape:  (42554, 256, 256, 3)
X_test Shape:  (10639, 256, 256, 3)


In [ ]:
# df_split = np.array_split(X_train, 20)
# nx_train = []
# for i in range(0,len(df_split)):
#     nx_train.append(df_split[i].astype('float32'))
#     df_split[i] = 0 
#     print(nx_train[i].shape)
    
# del df_split

In [ ]:
#X_train = np.hstack(nx_train[0], nx_train[1])

In [10]:
input_shape = (img_rows, img_cols, channels)

print("Normalizing Data")
X_train = X_train.astype('float16')
X_test = X_test.astype('float16')

X_train /= 255
X_test /= 255

Normalizing Data


In [11]:
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
print("y_train Shape: ", y_train.shape)
print("y_test Shape: ", y_test.shape)

y_train Shape:  (42554, 2)
y_test Shape:  (10639, 2)


In [16]:
print("Training Model")

model = cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size,nb_classes, nb_gpus=1)

print("Predicting")
y_pred = model.predict(X_test)

score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Precision: ", precision)
print("Recall: ", recall)

save_model(model=model, score=recall, model_name="DR_Two_Classes")
print("Completed")

Training Model
Model flattened out to:  (None, 438048)
Train on 34043 samples, validate on 8511 samples
Epoch 1/30
34043/34043 [==============================] - 306s 9ms/step - loss: 0.7602 - acc: 0.5086 - val_loss: 0.6919 - val_acc: 0.5277
Epoch 2/30
34043/34043 [==============================] - 300s 9ms/step - loss: 0.6978 - acc: 0.5151 - val_loss: 0.6942 - val_acc: 0.5277
Epoch 3/30
34043/34043 [==============================] - 301s 9ms/step - loss: 0.6937 - acc: 0.5173 - val_loss: 0.6920 - val_acc: 0.5277
Predicting
Test score: 0.6921419195521986
Test accuracy: 0.5245793777834776
Precision:  0.5245793777610678
Recall:  1.0
Saving Model
Completed
